In [8]:
# https://github.com/ahmedfgad/CNNGenetic

In [1]:
import numpy as np
import pygad
from tensorflow import keras
import os
from PIL import Image

In [2]:
file_paths = [os.path.join("training_set/benign", filename) for filename in os.listdir("training_set/benign") if "mask" not in filename] + [os.path.join("training_set/malignant", filename) for filename in os.listdir("training_set/malignant") if "mask" not in filename]
train_labels = [0.0 for filename in os.listdir("training_set/benign") if "mask" not in filename] + [1.0 for filename in os.listdir("training_set/malignant") if "mask" not in filename]

train_data = []
for path in file_paths:
    img = Image.open(path)
    img = img.resize((224,224))
    train_data.append(np.asarray(img))

train_data = np.array(train_data)
train_labels = np.array(train_labels) #keras.utils.to_categorical(train_labels, 2)

In [3]:
file_paths = [os.path.join("testing_set/benign", filename) for filename in os.listdir("testing_set/benign") if "mask" not in filename] + [os.path.join("testing_set/malignant", filename) for filename in os.listdir("testing_set/malignant") if "mask" not in filename]
test_labels = [0.0 for filename in os.listdir("testing_set/benign") if "mask" not in filename] + [1.0 for filename in os.listdir("testing_set/malignant") if "mask" not in filename]

test_data = []
for path in file_paths:
    img = Image.open(path)
    img = img.resize((224,224))
    test_data.append(np.asarray(img))

test_labels = keras.utils.to_categorical(test_labels, 2)

In [4]:
print(train_data.shape)
print(train_labels.shape)

(417, 224, 224, 3)
(417,)


In [5]:
# define model

def fitness_function(ga, solution, sol_idx):

    model = keras.Sequential([
        keras.Input(shape= (224, 224, 3)),
        keras.layers.ZeroPadding2D(padding= 1),
        keras.layers.Conv2D(solution[0], kernel_size= 3, activation = "relu"),
        keras.layers.MaxPooling2D(2),
        keras.layers.ZeroPadding2D(padding= 1),
        keras.layers.Conv2D(solution[1], kernel_size= 3, activation= "relu"),
        keras.layers.MaxPooling2D(2),
        keras.layers.ZeroPadding2D(padding= 1),
        keras.layers.Conv2D(solution[2], kernel_size= 3, activation= "relu"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(solution[3]),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1),
        keras.layers.Activation(keras.activations.sigmoid)
    ])

    # optimizer Adam, loss BCE, 
    model.compile(optimizer= "adam", loss= "binary_crossentropy", metrics= ["accuracy"])

    model.fit(train_data, train_labels, epochs= 1, batch_size= 100, verbose = 0)

    _, accuracy = model.evaluate(train_data, train_labels)

    return accuracy

In [6]:
def on_generation(ga):
    print("generation complete")

In [7]:
# create GA

ga_model = pygad.GA(
    num_generations= 36, # can raise
    num_parents_mating= 2, # can alter
    fitness_func= fitness_function,
    on_generation= on_generation,
    sol_per_pop= 10, # can raise
    num_genes= 4,
    gene_type= int,
    gene_space= {'low': 1, 'high': 224}, # can alter
    init_range_low= 20, # can alter
    init_range_high= 150, # can alter
    mutation_percent_genes= 30, # can alter
)

ga_model.run()






14/14 [==============================] - 2s 100ms/step - loss: 10.6694 - accuracy: 0.7122
generation complete
14/14 [==============================] - 2s 107ms/step - loss: 168.7573 - accuracy: 0.2878
generation complete
14/14 [==============================] - 16s 1s/step - loss: 162.6254 - accuracy: 0.7122
generation complete
14/14 [==============================] - 18s 1s/step - loss: 94.6118 - accuracy: 0.7122
generation complete
14/14 [==============================] - 9s 623ms/step - loss: 47.6154 - accuracy: 0.7122
generation complete
14/14 [==============================] - 4s 263ms/step - loss: 4.8678 - accuracy: 0.7122
generation complete
14/14 [==============================] - 4s 260ms/step - loss: 6.0590 - accuracy: 0.7170
generation complete
14/14 [==============================] - 2s 126ms/step - loss: 38.1125 - accuracy: 0.2878
generation complete
14/14 [==============================] - 2s 133ms/step - loss: 0.5182 - accuracy: 0.7410
generation complete
14/14 [===

KeyboardInterrupt: 

In [9]:
solution = ga_model.best_solution()

print(solution[0])
print(solution[1])

14/14 [==============================] - 4s 238ms/step - loss: 179.7827 - accuracy: 0.2878
[ 18 199 178  70]
0.8105515837669373
